# Hierarchical Cluster Analysis (HCA)

## Introduction

Hierarchical Cluster Analysis (HCA) is a method for grouping similar objects into clusters based on their similarity. Unlike flat clustering methods like k-means, HCA creates a hierarchy of nested clusters that can be visualized using a dendrogram.

There are two main types of hierarchical clustering:

1. **Agglomerative (bottom-up)**: Each data point starts as its own cluster, and clusters are merged iteratively.
2. **Divisive (top-down)**: All data points start in one cluster, and clusters are split iteratively.

### Applications of HCA
HCA is widely used in various fields manily for trend analysis, including:
- **Bioinformatics & Genomics**: Identifying gene expression patterns and evolutionary relationships.
- **Market Segmentation**: Grouping customers based on purchasing behavior.
- **Image Processing**: Clustering pixels with similar attributes for object recognition.
- **Environmental Science**: Categorizing chemical pollutants based on their similarities.
- **Social Sciences**: Analyzing survey responses to detect behavioral patterns.

Here we are going to focuse on the agglomerative approach as it is the most commonly used approach for HCA implementation. 

### Steps of HCA
1. Compute the pairwise distances between all points.
2. Identify the two closest clusters and merge them.
3. Update the distance matrix.
4. Repeat steps 2 and 3 until all points are in one cluster.
5. Represent the results using a **dendrogram**.

## How 

To be able to go through different steps of HCA algorithm we first need to generate some synthetic data that we can use. Please note that the generated dataset is one dimensional for ease of explanation and visualization. The same approach can be expanded to multiple dimensions.

In [ ]:
using ACS

ACS.Random.seed!(42)
X = vcat(rand(5, 2) .+ 1, rand(5, 2) .+ 4)  # Two distinct clusters
scatter(X[:,1],X[:,2],label="data")
xlabel!("1st Dim")
ylabel!("2nd Dim")

Let's ignore the second dimension of *X* for the moment and only focus on the 1st dimension. 

In [ ]:
X1 = X[:,1];

As we are using working with the agglomerative approach, we start with assuming that all the data points are individual clusters. So in this case we have 10 starting clusters. 

### Step 1: pair-wise distance calculations 

In unidimensional space the distance between two datapoints are defined by the below equation: 

$ 
d_{m,n} = x_n - x_m \\
m,n \in \mathbb{N}^+ \\
n \geq m
$


### E1: 

Let's write a function to calculate 1D distances between all the points in the *X1*.

In [ ]:
function dist(data)
    d = zeros(size(data,1),size(data,1)) #initialize distance matrix
    for i in 1:size(data,1) #loop over all data points
        for j in 1:size(data,1) #loop over all data points
            d[i,j] = abs(data[i]-data[j]) #calculate absolute distance
        end
    end
    return d    

end 

d = dist(X1)

### E2: 

Why do we get a square matrix?

> Answer to E2
> 
> The square matrix is the result of pair-wise distance calculations as you calculate the distances for all possible combinations of indices. The diagonal is zero as the distance of each point with itslef is zero. 


### E3: 

How do we form the first cluster? 

> Answer to E3:
>
> The smallest distances that is not on the diagonal gives you the coordinates of the first cluster. To do these calculations, you can first replace the diagonal with *Inf*, enabling you to search for the smallest non-zero distances. We can see that the fourth and first data-points can form the first cluster. 
>
> 

In [ ]:
d_ = deepcopy(d)
d_[d_ .== 0] .= Inf
d_

idx = argmin(d_)

In [ ]:
scatter(X1, X[:,2], label="data")
scatter!([X1[idx[1]] X1[idx[2]]], [X[:,2][idx[1]] X[:,2][idx[2]]], label="closest pair", color="red")
#xlims!(1, 1.5)
xlabel!("1st Dim")
ylabel!("2nd Dim")

<div class="alert alert-block alert-warning">
<b>Tip:</b> It is very important to use deepcopy(-) to make sure that the original distance matrix (d) is not modified during the agglomerative clustering.
</div>

### E4:

What is the next step? How can we move forward?

> Answer to E4:
>
> We replace the data points clustered together with their average, which should sit in between them. 
> 

In [ ]:
X_c = deepcopy(X)
X_c[idx[1],:] = (X[idx[1],:] .+ X[idx[2],:])./2
X_c[idx[2],:] = (X[idx[1],:] .+ X[idx[2],:])./2

scatter(X[:,1],X[:,2],label="original data")
scatter!(X_c[:,1],X_c[:,2],label="new data",color="red")
xlabel!("1st Dim")
ylabel!("2nd Dim")

### E5: 

Let's find the next cluster.

In [ ]:
d = dist(X_c[:,1])
d[d .== 0] .= Inf
idx = argmin(d)
scatter(X_c[:,1], X_c[:,2], label="data 2nd itr")
scatter!([X_c[:,1][idx[1]] X_c[:,1][idx[2]]], [X_c[:,2][idx[1]] X_c[:,2][idx[2]]], label="closest pair", color="red")
#xlims!(1, 1.5)
xlabel!("1st Dim")
ylabel!("2nd Dim")

If you repeat this process several times you will eventually end up with all the data points being agglomerated into one single large cluster.  

### E6: 

How do you know that all the data points have been clustered?

> Answer to E6
>
> Over each iteration the data points clustered together have a zero distance. Consequently, once all data points are clustered into one, all distances will be zero. 
> 

### E7: 

So far we only looked at a one dimensional dataset to be clustered. What if we want to do this with a multi-dimensional data (e.g. *X* rather than *X1*)?

> Answer to E7: 
>
> We need to modify the distance calculation function to allow more than one dimension. Please note for the time being we will only focus on the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance). There are many other distance metrics that will be discussed later on. 
> 

In [ ]:
function dist_euc(data)
    d = zeros(size(data,1),size(data,1)) #initialize distance matrix
    for i in 1:size(data,1) #loop over all data points
        for j in 1:size(data,1) #loop over all data points
            d[i,j] = sqrt(sum((data[i,:] .- data[j,:]).^2)) #calculate Euclidean distance
        end
    end
    return d    

end 

d = dist_euc(X)

### E8: 

What are the first data points to cluster together with the new distance matrix?

In [ ]:
d_ = deepcopy(d)
d_[d_ .== 0] .= Inf
idx = argmin(d_)

scatter(X[:,1], X[:,2], label="data")
scatter!([X[:,1][idx[1]] X[:,1][idx[2]]], [X[:,2][idx[1]] X[:,2][idx[2]]], label="closest pair", color="red")
#xlims!(1, 1.5)
xlabel!("1st Dim")
ylabel!("2nd Dim")

As you can see in the above plot, depending on the number of dimensions used the starting cluster may be different. 

The package [*Clustering.jl*](https://juliastats.org/Clustering.jl/stable/hclust.html), available via *ACS.jl* gives to access to a set of functions that are very useful for your cluster analysis. 

## HCA in practice: 

The main function to be used for HCA is [*hclust(-)*](https://juliastats.org/Clustering.jl/stable/hclust.html#Clustering.hclust). This function outputs the generated clusters.  

<div class="alert alert-block alert-info">
<b>Note:</b> Use blue boxes (alert-info) for tips and notes. 
It should be noted that the function hclust takes as input your distance matrix. Thus all clusters start with Euclidean distance matrix. However, you can select a different distance metrics for grouping on two clusters. 
</div>

### E9: 

Let's perform HCA on the matrix X. 

In [ ]:
d = dist_euc(X) # Euclidean distance matrix
result = hclust(d, linkage=:single) # Single linkage clustering
plot(result, labels=1:size(X,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Single Linkage Clustering") # Plot the dendrogram

The results of HCA is usually displayed via [dendrogram](https://en.wikipedia.org/wiki/Dendrogram), figure above. Sometimes a combination of dendrogram and heatmaps are used for analyzing the results of HCA. The output of HCA has two attributes "order" and "merges". The attribute "order" gives you the order of the clusters from left to right - in example above from 3 to 10. The "merges" shows what measurements were merged and at what level. 

```julia 

result.order # Clusters
result.merges # Merges

```

In [ ]:
X_ = X[result.order,:] # Reorder the data matrix
p1 = heatmap(X_, color=:jet, aspect_ratio=1, title="Single Linkage Clustering",cbar= :none, yaxis = false) # Plot the heatmap
xlims!(0, 3) # Set the x-axis limits
p2 = plot(result, labels=1:size(X,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :horizontal, linewidth=1, title="Single Linkage Clustering") # Plot the dendrogram
plot(p1, p2, layout=(1,2), size=(800,400)) # Combine the heatmap and dendrogram

You can also cut the dendrogram to generate a specific number of clusters. This is done via the function *cutree()* via *ACS.jl/Clustering.jl*. 

In [ ]:
idx_c = cutree(result, k=2) # Cut the dendrogram into 2 clusters
scatter(X[:,1], X[:,2], label=["Cluster 1" "Cluster 2"], group = idx_c, color=idx_c, legend=:bottomright, title="Single Linkage Clustering") # Plot the data points with cluster labels
xlabel!("1st Dim")
ylabel!("2nd Dim")  

In [ ]:
idx_c = cutree(result, k=3) # Cut the dendrogram into 2 clusters
scatter(X[:,1], X[:,2], label=["Cluster 1" "Cluster 2" "Cluster 3"], group = idx_c, color=idx_c, legend=:bottomright, title="Single Linkage Clustering") # Plot the data points with cluster labels
xlabel!("1st Dim")
ylabel!("2nd Dim")  

### E10: 

How can we use another distance metrics, for example Jaccard? 

> Answer to 10
>
> You can use the function *pairwise(-)* from *ACS.jl/Distances.jl* with the keyword Mahalanobis to perform these calculations.
> 

In [ ]:
d = pairwise(Jaccard(), X,dims = 1) # Mahalanobis distance matrix

In [ ]:
r_c = hclust(d, linkage=:single) # Single linkage clustering
plot(r_c, labels=1:size(X,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Single Linkage Clustering") # Plot the dendrogram


As you can see the composition of the two large clusters did not change while the immediate connections in some cases are slightly different from the previously generated clusters based on Euclidean distances. 

### E11:

Let's generate a more complex dataset and assess the impact of different linkages on the final clusters. The available linkage methods are the following:

- **:single**: distance between the closest points of two clusters.
- **:average**: use the mean distance between any of the cluster members 
- **:complete**: distance between the farthest points of two clusters.
- **:ward**: the distance is the increase of the average squared distance of a point to its cluster centroid after merging the two clusters.

```julia

Z = rand(15, 2) * 10 # data generation

```

In [ ]:
Z = rand(15, 2) * 10
d = pairwise(Euclidean(), Z,dims = 1) # Euclidean distance matrix
r_c_s = hclust(d, linkage=:single) # Single linkage clustering
r_c_a = hclust(d, linkage=:average) # Average linkage clustering
r_c_c = hclust(d, linkage=:complete) # Complete linkage clustering
r_c_w = hclust(d, linkage=:ward) # Ward linkage clustering

p1 = plot(r_c_s, labels=1:size(Z,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Single Linkage Clustering") # Plot the dendrogram
p2 = plot(r_c_a, labels=1:size(Z,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Average Linkage Clustering") # Plot the dendrogram
p3 = plot(r_c_c, labels=1:size(Z,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Complete Linkage Clustering") # Plot the dendrogram
p4 = plot(r_c_w, labels=1:size(Z,1), color_threshold=0.5, show_leaf_labels=true,
 show_inner_nodes=true, show_root=true, linecolor=:black, orientation = :v, linewidth=1, title="Ward Linkage Clustering") # Plot the dendrogram
plot(p1, p2, p3, p4, layout=(2,2), size=(800,800)) # Combine the dendrograms    

As you can see, changing the merging metrics could drastically change the structure of the generated clusters. 

### E12: 

How do you decide which linkage is the best?

> Answer to E12:
>
> Generally for any hyperparameter optimization, you should have some labeled data, enabling you to assess the quality of the generated clusters. This also can be done via expert knowledge. For example, if you a priori know that two samples are supposed to be very similar to each other (e.g. two replicates) the linkage approach that cluster them with no intermediates is more adequate to answer your scientific question. 

## Extensive Exercise: 

Use HCA for the analysis of Iris dataset. Assess which linkage approach is performing the best and finally plot the results. 

In [ ]:
data = dataset("datasets", "iris")
X_iris = Matrix(data[:, 1:4])  # Selecting only numerical features

"""
## 2. Computing Pairwise Distances
We compute the pairwise distance matrix using Euclidean distance.
"""

dist_iris = pairwise(Euclidean(), X_iris, dims=1)

"""
## 3. Performing Hierarchical Clustering using Different Linkage Methods
We compare Single, Complete, and Average linkage methods.
"""

linkage_single = hclust(dist_iris, linkage=:single)
linkage_complete = hclust(dist_iris, linkage=:complete)
linkage_average = hclust(dist_iris, linkage=:average)
linkage_ward = hclust(dist_iris, linkage=:ward)

"""
## 4. Visualizing the Dendrograms
We plot the dendrograms for each linkage method.
"""

p1 = plot(linkage_single, xticks=false, title="HCA with Single Linkage", xlabel="Samples", ylabel="Distance")
p2 = plot(linkage_complete, xticks=false, title="HCA with Complete Linkage", xlabel="Samples", ylabel="Distance")
p3 = plot(linkage_average, xticks=false, title="HCA with Average Linkage", xlabel="Samples", ylabel="Distance")
p4 = plot(linkage_ward, xticks=false, title="HCA with Ward Linkage", xlabel="Samples", ylabel="Distance")
plot(p1, p2, p3, p4, layout=(2, 2), size=(800, 800))


In [ ]:
"""
## 5. Evaluating Clustering Performance
To assess the clustering performance, we compare the results to the actual species labels using Adjusted Rand Index (ARI).
"""

# Extract cluster assignments
clusters_single = cutree(linkage_single, k=3)
clusters_complete = cutree(linkage_complete, k=3)
clusters_average = cutree(linkage_average, k=3)
clusters_ward = cutree(linkage_ward, k=3)

# True labels
true_labels = [data.Species[i] == "setosa" ? 1 : data.Species[i] == "versicolor" ? 2 : 3 for i in 1:size(data, 1)]

# Compute Adjusted Rand Index (ARI)
ari_single = ACS.randindex(clusters_single, true_labels)
ari_complete = ACS.randindex(clusters_complete, true_labels)
ari_average = ACS.randindex(clusters_average, true_labels)
ari_ward = ACS.randindex(clusters_ward, true_labels)

println("ARI for Single Linkage: ", ari_single)
println("ARI for Complete Linkage: ", ari_complete)
println("ARI for Average Linkage: ", ari_average)
println("ARI for Ward Linkage: ", ari_ward)